In [1]:
import os
import time
import logging
import pandas as pd
import torch
import torch.nn.functional as F
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from dateutil import parser

/home/sunkari/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
print(torch.__version__)

2.6.0+cu124


In [3]:
import os
import time
import logging
import torch
import torch.nn.functional as F
import pandas as pd
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from dateutil import parser

# =====================================================
LOG_FILE = "processing_log.txt"
logging.basicConfig(
    filename=LOG_FILE,
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S"
)

def log(msg, level="info"):
    tqdm.write(msg)
    if level == "error":
        logging.error(msg)
    elif level == "warning":
        logging.warning(msg)
    else:
        logging.info(msg)


# =====================================================
def normalize_date_column(df):
    def parse_date_safe(x):
        try:
            return parser.parse(str(x), dayfirst=False)
        except Exception:
            try:
                return parser.parse(str(x), dayfirst=True)
            except Exception:
                return None

    df["Date"] = df["Date"].apply(parse_date_safe)
    df = df.dropna(subset=["Date"])
    df = df.sort_values("Date").reset_index(drop=True)
    return df


def split_headlines(text):
    if pd.isna(text):
        return []
    return [t.strip() for t in str(text).split('|') if t.strip()]


# =====================================================
DATA_DIR = "/home/sunkari/Stock_price_predictor/Dataset"
OUTPUT_DIR = "./Processed"
os.makedirs(OUTPUT_DIR, exist_ok=True)

TRAIN_RATIO = 0.8  # 80% train, 20% test

# =====================================================
MODEL_NAME = "yiyanghkust/finbert-tone"
log(f"🔹 Loading model: {MODEL_NAME}")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)
model.eval()
log("✅ Model loaded successfully.")


# =====================================================
def get_sentiment_scores(texts):
    if len(texts) == 0:
        return [0.0, 1.0, 0.0]  # Neutral default
    inputs = tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )
    with torch.no_grad():
        outputs = model(**inputs)
        probs = F.softmax(outputs.logits, dim=-1)
    return probs.mean(dim=0).numpy().tolist()


# =====================================================
start_time_total = time.time()

for file in os.listdir(DATA_DIR):
    if not file.endswith(".csv"):
        continue
    start_time = time.time()
    company_path = os.path.join(DATA_DIR, file)
    log(f"\n🔍 Processing {file} ...")

    try:
        df = pd.read_csv(company_path)
        df.columns = df.columns.str.strip()
        log(f"📂 Loaded file with {len(df)} rows.")

        df = normalize_date_column(df)
        log(f"🗓️ Normalized dates, {len(df)} rows remain after cleaning.")

        # Split headlines into lists
        df["Headline_List"] = df["Headlines"].apply(split_headlines)

        # Compute daily sentiment
        sentiments = []
        for headlines in tqdm(df["Headline_List"].tolist(), desc=f"Sentiment {file}"):
            try:
                probs = get_sentiment_scores(headlines)
                sentiments.append(probs)
            except Exception as e:
                log(f"⚠️ Error processing headlines: {headlines[:3]}... | {e}", "warning")
                sentiments.append([0.0, 1.0, 0.0])  # default neutral

        sentiments = pd.DataFrame(sentiments, columns=["negative", "neutral", "positive"])
        df = pd.concat([df, sentiments], axis=1)

        # =====================================================
        # ✂️ Split into Train and Test
        # =====================================================
        split_idx = int(len(df) * TRAIN_RATIO)
        train_df = df.iloc[:split_idx].reset_index(drop=True)
        test_df = df.iloc[split_idx:].reset_index(drop=True)

        # Save processed versions
        base_name = os.path.splitext(file)[0]
        train_out = os.path.join(OUTPUT_DIR, f"{base_name}_train.csv")
        test_out = os.path.join(OUTPUT_DIR, f"{base_name}_test.csv")

        train_df.to_csv(train_out, index=False)
        test_df.to_csv(test_out, index=False)

        elapsed = time.time() - start_time
        log(f"✅ Saved train/test split for {file} | Train={len(train_df)}, Test={len(test_df)} | ⏱️ {elapsed:.2f}s")

    except Exception as e:
        log(f"❌ Error processing {file}: {e}", "error")

total_time = time.time() - start_time_total
log(f"\n🏁 All files processed successfully in {total_time/60:.2f} minutes.")

🔹 Loading model: yiyanghkust/finbert-tone
✅ Model loaded successfully.

🔍 Processing XOM_stock_gdelt_final.csv ...
📂 Loaded file with 1003 rows.
🗓️ Normalized dates, 1003 rows remain after cleaning.


Sentiment XOM_stock_gdelt_final.csv: 100%|██████████| 1003/1003 [03:05<00:00,  5.40it/s]


✅ Saved train/test split for XOM_stock_gdelt_final.csv | Train=802, Test=201 | ⏱️ 185.98s

🔍 Processing MSFT_stock_gdelt_final.csv ...
📂 Loaded file with 1003 rows.
🗓️ Normalized dates, 1003 rows remain after cleaning.


Sentiment MSFT_stock_gdelt_final.csv: 100%|██████████| 1003/1003 [03:40<00:00,  4.56it/s]


✅ Saved train/test split for MSFT_stock_gdelt_final.csv | Train=802, Test=201 | ⏱️ 220.27s

🔍 Processing V_stock_gdelt_final.csv ...
📂 Loaded file with 1003 rows.
🗓️ Normalized dates, 1003 rows remain after cleaning.


...

✅ Saved train/test split for TSLA_stock_gdelt_final.csv | Train=802, Test=201 | ⏱️ 273.02s

🔍 Processing JPM_stock_gdelt_final.csv ...
📂 Loaded file with 1003 rows.
🗓️ Normalized dates, 1003 rows remain after cleaning.


Sentiment JPM_stock_gdelt_final.csv: 100%|██████████| 1003/1003 [03:39<00:00,  4.57it/s]


✅ Saved train/test split for JPM_stock_gdelt_final.csv | Train=802, Test=201 | ⏱️ 219.45s

🔍 Processing AAPL_stock_gdelt_final.csv ...
📂 Loaded file with 1003 rows.
🗓️ Normalized dates, 1003 rows remain after cleaning.


Sentiment AAPL_stock_gdelt_final.csv: 100%|██████████| 1003/1003 [03:02<00:00,  5.51it/s]


✅ Saved train/test split for AAPL_stock_gdelt_final.csv | Train=802, Test=201 | ⏱️ 182.31s

🏁 All files processed successfully in 34.62 minutes.
